In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 前処理とデータ準備

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,))
                                ])
train_dataset = datasets.MNIST(root='./data',train=True,transform=transform,download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# データローダーの作成
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 9912422/9912422 [00:11<00:00, 861380.81it/s] 


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 28881/28881 [00:00<00:00, 144207.79it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 1648877/1648877 [00:03<00:00, 416298.51it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw


# CNNモデルの定義

In [3]:
class CNN(nn.Module):# pytorchの基本モジュールを使うために継承
    def __init__(self):
        super(CNN, self).__init__()
        # 1つ目の畳み込み層: 1入力チャンネル（グレースケール）、32出力チャンネル、カーネルサイズ3x3
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # 2つ目の畳み込み層: 32入力チャンネル、64出力チャンネル
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # プーリング層: 2x2のマックスプーリング
        self.pool = nn.MaxPool2d(2, 2)
        # ドロップアウト: 過学習防止のために50%を無効化
        self.dropout = nn.Dropout(0.25)
        # 全結合層1: 入力3136、出力128
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # 7x7はプーリング後のサイズ
        # 全結合層2: 出力10（クラス数）
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # 畳み込み1 -> ReLU -> プーリング
        x = self.pool(F.relu(self.conv1(x)))
        # 畳み込み2 -> ReLU -> プーリング
        x = self.pool(F.relu(self.conv2(x)))
        # 平坦化
        x = x.view(-1, 64 * 7 * 7)
        # 全結合1 -> ReLU -> ドロップアウト
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        # 全結合2 -> ソフトマックス
        x = self.fc2(x)
        return x

# モデルのインスタンス化と損失関数定義・最適化

In [4]:
# モデルのインスタンス化
model = CNN()

# 損失関数と最適化手法
criterion = nn.CrossEntropyLoss()  # 多クラス分類にクロスエントロピーを使用
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 学習ループ定義

In [5]:
# トレーニング関数
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # モデルを訓練モードに
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # 勾配を初期化
        output = model(data)  # フォワードパス
        loss = criterion(output, target)  # 損失計算
        loss.backward()  # バックプロパゲーション
        optimizer.step()  # 最適化ステップ
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# テストループ定義

In [6]:
# テスト関数
def test(model, device, test_loader):
    model.eval()  # モデルを評価モードに
    test_loss = 0
    correct = 0
    with torch.no_grad():  # テスト時には勾配を計算しない
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # バッチごとの損失
            pred = output.argmax(dim=1, keepdim=True)  # 最大値のインデックスが予測
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

# モデルのトレーニングとテスト

In [7]:
# GPUが使用可能なら使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# エポック数
num_epochs = 10

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300625
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.251307
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.258510
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.198069
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.147742
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.124912
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.073015
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.039745
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.084744
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.057966

Test set: Average loss: 0.0007, Accuracy: 9830/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.008916
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.052192
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.012884
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.118045
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.036073
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.140519
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.086967
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.090403
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.069874
T

# モデルの保存

In [13]:
import csv

# モデルの全てのパラメータを取り出す
model_params = model.state_dict()

# CSVに保存（データの一部を切り捨て）
with open('mnist_cnn_weights.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # ヘッダー行を追加（パラメータ名、形状、データの一部）
    writer.writerow(["Parameter Name", "Shape", "Sample Values (First 10)"])
    
    for key, value in model_params.items():
        # GPU上にあるテンソルをCPUに移動してからnumpyに変換
        value_cpu = value.cpu().numpy()
        
        # 各パラメータの先頭10個の値だけを保存（1次元に変換）
        flat_values = value_cpu.flatten()[:10]  # 先頭10個を取得
        
        # パラメータ名、形状、データの一部をCSVに保存
        writer.writerow([key, value_cpu.shape, flat_values.tolist()])


In [20]:
from PIL import Image
import os
import torch
from torchvision import transforms

# 複数画像の手書き数字を推論する関数
def predict_multiple_digits(model, image_paths, device):
    model.eval()  # モデルを評価モードに
    predictions = []  # 結果を保存するリスト

    # 画像の前処理設定
    transform = transforms.Compose([
        transforms.Resize((28, 28)),  # MNISTのサイズにリサイズ
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # 画像をループ処理
    for image_path in image_paths:
        # 画像を読み込む
        image = Image.open(image_path).convert('L')  # グレースケール画像に変換

        # 画像の前処理
        image = transform(image).unsqueeze(0)  # バッチサイズ1を追加

        # 画像をデバイスに転送
        image = image.to(device)

        # 勾配を計算しない設定で推論
        with torch.no_grad():
            output = model(image)
            pred = output.argmax(dim=1)  # 予測されたクラスを取得
            predictions.append((os.path.basename(image_path), pred.item()))  # 画像名と予測結果をリストに保存

    return predictions  # 予測結果を返す

# 画像ファイルがあるディレクトリのパス
image_dir = 'image_directory'  # 画像が保存されているディレクトリのパス

# ディレクトリ内の画像ファイルのパスを取得
image_paths = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith(('png', 'jpg', 'jpeg'))]

# 複数画像の予測を実行
predictions = predict_multiple_digits(model, image_paths, device)

# 予測結果を「画像名：予測した数字」の形式で表示
for image_name, predicted_digit in predictions:
    print(f'{image_name}: {predicted_digit}')


1.jpg: 1
3.png: 6
5.png: 5
9.jpg: 3
